In [ ]:
def get_absolute_path(*relative_parts):
    import os
    if 'dbutils' in globals():
        # dbutils is available to the entry point when run in Databricks
        base_dir = os.path.dirname(dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()) # type: ignore
        path = os.path.normpath(os.path.join(base_dir, *relative_parts))
        return path if path.startswith("/Workspace") else "/Workspace" + path
    else:
        return os.path.join(*relative_parts)


# Ingest sales data

Simulate an input port

In [25]:
import pandas as pd
from helpers.spark import get_spark

sales = get_absolute_path("..", "fixtures", "sales.csv")

dtype = {"sku": str}

df = pd.read_csv(sales, dtype=dtype)

df['order_timestamp'] = pd.to_datetime(df['order_timestamp'])
df['processing_timestamp'] = pd.to_datetime(df['processing_timestamp'])

display(df)

sdf = get_spark().createDataFrame(df)
sdf.write.format("delta").mode("overwrite").saveAsTable("acme.stock_last_sales.input_sales")


,order_id,customer_id,sku,order_timestamp,processing_timestamp
0,0fff1c4b-bd7e-4c18-963d-f2246b873ade,deca697a-a826-4153-a087-e54c78cdb0ee,400762254142559,2024-03-15 19:31:48.477718,2024-03-17 19:31:48.501565
1,9aa9ecf9-d3b5-41bc-810a-09a3c8aedb8a,77f327de-adea-4a10-acee-3ed84a3aff02,401864661022360,2024-03-05 19:31:48.477766,2024-03-05 19:31:48.501570
2,4f21fec4-bc88-4470-aea3-c2008efc41b8,2a256be1-2d16-4a5b-b5eb-01f6a4c6b56f,404440578943492,2024-03-27 19:31:48.477777,2024-03-01 19:31:48.501574
3,c3a756a0-d213-4ef0-a9e2-13020901ada1,cafc281e-04c1-4035-98b7-444d0c183f71,403594298394150,2024-03-08 19:31:48.477785,2024-03-06 19:31:48.501577
4,f06e44ef-7ce4-44e2-b2cf-ac15cdfd541f,a6e82cd1-9b0a-48b7-9260-d2da777feebc,404016673836668,2024-03-02 19:31:48.477793,2024-02-29 19:31:48.501581
...,...,...,...,...,...
2995,bc09e313-62ee-47ec-84fb-97175c090249,123dd415-dbd7-4a1b-9603-eca883623b90,405004895686435,2024-03-19 19:31:48.501538,2024-03-17 19:31:48.513217
2996,fe810bce-eeeb-4269-abba-e1fc57a3511a,9fb428d5-86cf-47bd-b9ec-f31387798ca5,404706507927581,2024-02-26 19:31:48.501542,2024-03-21 19:31:48.513221
2997,5fba5001-cd2f-414e-8497-e87a7d27cca4,bceb69c7-2852-4df0-b03f-a3d36f237a5d,400025190542666,2024-03-20 19:31:48.501545,2024-03-16 19:31:48.513224
2998,0507000b-86ec-4391-a5ef-db207023bfc9,8c269455-c00e-4974-8bce-70f056153a16,404354896577312,2024-03-18 19:31:48.501549,2024-03-04 19:31:48.513227


# Ingest stock data

Simulate an input port

In [26]:
import pandas as pd
import os


sales = get_absolute_path("..", "fixtures", "stock.csv")

dtype = {"sku": str}
df = pd.read_csv(sales, dtype=dtype)

df['updated_timestamp'] = pd.to_datetime(df['updated_timestamp'])
df['processing_timestamp'] = pd.to_datetime(df['processing_timestamp'])

display(df)

sdf = spark.createDataFrame(df)
sdf.write.format("delta").mode("overwrite").saveAsTable("acme.stock_last_sales.input_stock")

,sku,quantity,updated_timestamp,processing_timestamp
0,402989803787501,26,2024-03-07 19:27:56.802952,2024-02-29 19:27:56.809050
1,404273656385451,539,2024-03-08 19:27:56.802967,2024-03-09 19:27:56.809054
2,401502677161258,501,2024-03-04 19:27:56.802970,2024-03-02 19:27:56.809058
3,402231140467550,369,2024-03-25 19:27:56.802972,2024-03-25 19:27:56.809061
4,404482135138781,698,2024-03-27 19:27:56.802973,2024-03-08 19:27:56.809064
...,...,...,...,...
1995,400610044010350,240,2024-02-29 19:27:56.809032,2024-02-26 19:27:56.816996
1996,403427472018516,358,2024-03-09 19:27:56.809035,2024-03-04 19:27:56.816998
1997,403579224511153,801,2024-02-29 19:27:56.809037,2024-03-17 19:27:56.817000
1998,400705739062843,530,2024-03-18 19:27:56.809039,2024-03-05 19:27:56.817002
